# Remove Arabic sentences

In [ ]:
import pandas as pd
import nltk
import re

In [ ]:
# Checking removing Arabic sentences
data = pd.read_csv('splitted_reviews_into_sentences_sID.csv') 
df = pd.DataFrame(data, columns=['r_id', 's_id','app_name','app_id', 'date', 'title', 'review', 'rating'])

In [ ]:
# Arabic review > df['review'][14614] 2760
# Review with Emoji > df['review'][727]
# Review with numbers > df['review'][726]
sent = df['review'][36]
print(sent)

In [ ]:
reviews_list = df['review'].tolist()
print(reviews_list)

In [ ]:
len(df)

In [ ]:
arabic_sent = 0
for x in range(len(reviews_list)):
    if re.match('[\u0621-\u064A\u0660-\u0669 ]+', reviews_list[x]):
        df.drop(x, inplace=True)
        arabic_sent = arabic_sent + 1
        print(x, reviews_list[x])
        
print("Number of dropped Arabic sentences:", arabic_sent)

In [ ]:
df = df.reset_index(drop=True)

In [ ]:
len(df)

In [ ]:
df['review'][36]

In [ ]:
table = {"r_id": [], "s_id": [], "app_name": [], "app_id": [], "date": [], "title": [], "review": [], "rating": []}

for x in range(len(df)):
    table["r_id"].append(df['r_id'][x])
    table["s_id"].append(df['s_id'][x])
    table["app_name"].append(df['app_name'][x])
    table["app_id"].append(df['app_id'][x])
    table["date"].append(df['date'][x])
    table["title"].append(df['title'][x])
    table["review"].append(df['review'][x])
    table["rating"].append(df['rating'][x])

In [ ]:
df=pd.DataFrame.from_dict(table,orient='index').transpose()
df.to_csv('reviews_without_Arabic_sentences.csv', index = False)
# This file has been taken to find sentences reference to the extracted features

#  If I want to group the sentences back as reviews

In [ ]:
df['review'] = df[['r_id', 's_id','app_name','app_id', 'date', 'title', 'review', 'rating']].groupby(['r_id'])['review'].transform(lambda x: ' '.join(x))

In [ ]:
df = df[['r_id','app_name','app_id', 'date', 'title', 'review', 'rating']].drop_duplicates(subset=None, keep='first', inplace=False, ignore_index=False)

In [ ]:
df = df.reset_index(drop=True)

In [ ]:
len(df)

In [ ]:
table = {"r_id": [], "app_name": [], "app_id": [], "date": [], "title": [], "review": [], "rating": []}

for x in range(len(df)):
    table["r_id"].append(df['r_id'][x])
    table["app_name"].append(df['app_name'][x])
    table["app_id"].append(df['app_id'][x])
    table["date"].append(df['date'][x])
    table["title"].append(df['title'][x])
    table["review"].append(df['review'][x])
    table["rating"].append(df['rating'][x])

In [ ]:
df=pd.DataFrame.from_dict(table,orient='index').transpose()
df.to_csv('reviews_without_Arabic_reviews_sentences.csv', index = False)

# Process sentiment scores

In [ ]:
dataS = pd.read_csv('sentences_sentistrength_results.csv') 
df_sentences = pd.DataFrame(dataS, columns=['reference_sentence', 'Positive','Negative'])

In [ ]:
sentence_sentiment = []
for x in range(len(df_sentences)):
    if df_sentences['Positive'][x] == abs(df_sentences['Negative'][x]):
        if df_sentences['Positive'][x] == 1:
            sentence_sentiment.insert(x, 0) 
        else:
            sentence_sentiment.insert(x, df_sentences['Negative'][x]) 
    elif df_sentences['Positive'][x] > abs(df_sentences['Negative'][x]):
        sentence_sentiment.insert(x, df_sentences['Positive'][x]) 
    else:
        sentence_sentiment.insert(x, df_sentences['Negative'][x]) 

In [ ]:
len(sentence_sentiment)

In [ ]:
table = {"sent_id": [], "sentence": [], "sentence_sentiment": []}

for x in range(len(df_sentences)):
    table["sent_id"].append(x)
    table["sentence"].append(df_sentences['reference_sentence'][x])
    table["sentence_sentiment"].append(sentence_sentiment[x])

In [ ]:
df=pd.DataFrame.from_dict(table,orient='index').transpose()
df.to_csv('sentences_sentiment_score.csv', index = False)

In [ ]:
df

In [ ]:
dataR = pd.read_csv('reviews_sentistrength_results.csv') 
df_reviews = pd.DataFrame(dataR, columns=['reference_review', 'Positive','Negative'])

In [ ]:
review_sentiment = []
for x in range(len(df_reviews)):
    if df_reviews['Positive'][x] == abs(df_reviews['Negative'][x]):
        if df_reviews['Positive'][x] == 1:
            review_sentiment.insert(x, 0) 
        else:
            review_sentiment.insert(x, df_reviews['Negative'][x]) 
    elif df_reviews['Positive'][x] > abs(df_reviews['Negative'][x]):
        review_sentiment.insert(x, df_reviews['Positive'][x]) 
    else:
        review_sentiment.insert(x, df_reviews['Negative'][x]) 

In [ ]:
table = {"review": [], "review_sentiment": []}

for x in range(len(df_reviews)):
    table["review"].append(df_reviews['reference_review'][x])
    table["review_sentiment"].append(review_sentiment[x])

In [ ]:
df=pd.DataFrame.from_dict(table,orient='index').transpose()
df.to_csv('reviews_sentiment_score.csv', index = False)

# Add r_id for SentiStrength files

In [ ]:
dataS = pd.read_csv('sentences_sentiment_score.csv') 
df_sentences = pd.DataFrame(dataS, columns=['sentence','sentence_sentiment'])

In [ ]:
dataRS = pd.read_csv('reference_sentences.csv') 
df_ref_sentences = pd.DataFrame(dataRS, columns=['r_id', 's_id'])

In [ ]:
combined_df = pd.concat([df_ref_sentences, df_sentences], axis=1, join='inner')

In [ ]:
combined_df.to_csv('sentences_sentiment_score_with_id.csv', index = False)

In [ ]:
dataR = pd.read_csv('reviews_sentiment_score.csv') 
df_reviews = pd.DataFrame(dataR, columns=['review','review_sentiment'])

In [ ]:
dataRR = pd.read_csv('reference_reviews.csv') 
df_ref_reviews = pd.DataFrame(dataRR, columns=['r_id'])

In [ ]:
combined_dfR = pd.concat([df_ref_reviews, df_reviews], axis=1, join='inner')

In [ ]:
combined_dfR.to_csv('reviews_sentiment_score_with_id.csv', index = False)

# Combine sentences and reviews in one file

In [ ]:
r_id_from_reviews = combined_dfR['r_id'].tolist()
r_id_from_sentences = combined_df['r_id'].tolist()
sentences_list = combined_df['sentence'].tolist()
reviews_list = combined_dfR['review'].tolist()

In [ ]:
combined_dfR

In [ ]:
combined_df

In [ ]:
# Combine reviews with sentences
table = {"r_id": [], "review": [], "review_sentiment": [], "s_id": [], "sentence": [], "sentence_sentiment": []}
counter = 0
for x in range(len(r_id_from_sentences)):
    for y in range(len(r_id_from_reviews)):
        if r_id_from_sentences[x] == r_id_from_reviews[y]:
            print("Sentence r_id: ", r_id_from_sentences[x], "equals review r_id: ", r_id_from_reviews[y])
            print("*Sentence", sentences_list[x])
            print("**Review", reviews_list[y])
            print("-------")
            counter = counter + 1
            table["r_id"].append(r_id_from_reviews[y])
            table["review"].append(reviews_list[y])
            table["review_sentiment"].append(combined_dfR['review_sentiment'][y])
            table["s_id"].append(combined_df['s_id'][x])
            table["sentence"].append(sentences_list[x])
            table["sentence_sentiment"].append(combined_df['sentence_sentiment'][x])
        

print("Reference reviews: ", counter)

In [ ]:
df=pd.DataFrame.from_dict(table,orient='index').transpose()
df.to_csv('reviews_sentences_sentiment_scores.csv', index = False)

In [ ]:
len(df)

In [ ]:
df

# Assign sentiment for features

In [ ]:
dataF = pd.read_csv('features_with_weight.csv') 
dfF = pd.DataFrame(dataF, columns=['extracted_features', 'meaningful', 'weight'])

In [ ]:
dataS = pd.read_csv('reviews_sentences_sentiment_scores.csv') 
dfS = pd.DataFrame(dataS, columns=['r_id', 'review', 'review_sentiment', 's_id', 'sentence', 'sentence_sentiment'])

In [ ]:
sentences_list = dfS['sentence'].str.lower().tolist()

In [ ]:
for x in range(len(sentences_list)):
    if re.search('costumer service', sentences_list[x]):
        print("Old:", sentences_list[x])
        string = sentences_list[x]
        sentences_list[x] = string.replace('costumer service', 'customer service')
        print("New:", sentences_list[x])

In [ ]:
# Add the meaningful features to the features list
features_list = []
weight = []
for x in range(len(dfF)):
    if dfF['meaningful'][x] == 1:
        features_list.insert(x, dfF['extracted_features'][x])
        weight.insert(x, dfF['weight'][x])

In [ ]:
# Find each features with its reference review/sentence to assign sentiment score
table = {"r_id": [], "review": [], "review_sentiment": [], "s_id": [], "sentence": [], "sentence_sentiment": [], "feature": [], "weight": []}
counter = 0

for x in range(len(features_list)):
    print("-------feature", features_list[x])
    for y in range(len(sentences_list)):
        if features_list[x] in sentences_list[y]:
            counter = counter + 1
            print("Feature:", features_list[x], "weight:", weight[x])
            print("Found in sentence ID #:", dfS['s_id'][y], "|| Sentence::", sentences_list[y])
            table["r_id"].append(dfS['r_id'][y])
            table["review"].append(dfS['review'][y])
            table["review_sentiment"].append(dfS['review_sentiment'][y])
            table["s_id"].append(dfS['s_id'][y])
            table["sentence"].append(sentences_list[y])
            table["sentence_sentiment"].append(dfS['sentence_sentiment'][y])
            table["feature"].append(features_list[x])
            table["weight"].append(weight[x])
        

print(counter)

In [ ]:
df=pd.DataFrame.from_dict(table,orient='index').transpose()
df.to_csv('final_file_with_sentiments.csv', index = False)

# Add feature frequency

In [ ]:
data = pd.read_csv('final_file_with_sentiments.csv') 
df = pd.DataFrame(data, columns=['r_id', 'review', 'review_sentiment', 's_id', 'sentence', 'sentence_sentiment', 'feature'])

# frequency regardless sentiment


In [ ]:
df['freq'] = df.groupby('feature')['feature'].transform('count')

In [ ]:
df

In [ ]:
# frequency regardless sentiment
df.to_csv('freq_regardless_sentiment.csv', index = False)

# frequency regarding feature and sentiment

In [ ]:
frequency_series = df.groupby(['review_sentiment', 'sentence_sentiment', 'feature']).size()

In [ ]:
new_df = frequency_series.to_frame(name = 'freq').reset_index()

In [ ]:
new_df

In [ ]:
new_df.to_csv('freq_regard_sentiment.csv', index = False)

In [ ]:
# grouped_df = df.groupby(["review_sentiment", "sentence_sentiment", "feature"]).size().reset_index(name="Frequency")

In [ ]:
# grouped_df.to_csv('grouped_file.csv', index = False)

In [ ]:
# This is old file without including reviews sentiment score
df.to_csv('features_sentiment_score_with_sent.csv', index = False)